In [5]:
# build custom bowtie index
# skip if already done

if [ ! -d "$index_file" ]; then
    lib_sequence=/home/xw2629/noncoding_translation/old-data/oligo-pool-design/XuebingWu-12K-120-oligo-pool.fa
    mkdir $outputFolder/N90_bowtie_index
    cd $outputFolder/N90_bowtie_index
    bowtie-build $lib_sequence N90bowtie

fi

mkdir: cannot create directory ‘/home/xw2629/noncoding_translation/noncoding-translation-code/pep30-bag6-ko/N90_bowtie_index’: File exists
Settings:
  Output files: "N90bowtie.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/xw2629/noncoding_translation/old-data/oligo-pool-design/XuebingWu-12K-120-oligo-pool.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN sett

In [13]:
# from FASTQ to counts 
 
rep=1 # or 2

if [ $rep == "1" ]
then
    fastqFolder=/media/rna/sequencing_data/jordan/Pep30-BAG6-KO-20211202 # rep 1
    samples="HEK293T-RATIO BA023E5-RATIO"
else
    fastqFolder=/media/rna/sequencing_data/jordan/Pep30-BAG6-KO-20221213 # rep 2
    samples="G H"
fi

index_file=$outputFolder/N90_bowtie_index/N90bowtie

outputFolder=/home/xw2629/noncoding_translation/noncoding-translation-code/pep30-bag6-ko

# unzip all fastq files
cd $fastqFolder
gunzip *.fastq.gz

cd $outputFolder

for sample in $samples # rep1: HEK293T-RATIO BA023E5-RATIO; rep2: G H
do
    for bin in HIGH MIDHIGH MIDLOW LOW # each sample sorted into 4 bins 
    do
        # each library: sample-bin
        #C-HIGH_S1_L001_R1_001.fastq.gz
        lib=$sample'-'$bin 
        echo $lib
        
        # each library has 4 fastq files, concat them
        all_files=
        for file in $fastqFolder/*
        do
            filename="$(basename -- $file)"
            if [[ $filename == *$lib*.fastq ]]
            then
                all_files=$file","$all_files
            fi
        done
                
        #count reads
        tag=
        if [ $sample == "G" ] || [ $sample == "HEK293T-RATIO" ]
        then
            tag="WT"
        else
            tag="KO"
        fi
        #echo $tag
        echo $all_files
        bowtie $index_file $all_files -v 3 --best -p 80 | cut -f 3 | sort | uniq -c | awk '{print $2"\t"$1}' > $tag-$bin-$rep.count 
    done
done

cd $fastqFolder
#gzip *.fastq
